In [2]:
import numpy as np
import math
import seaborn as sns
from matplotlib import pyplot as plt


The [heatindex](https://de.wikipedia.org/wiki/Hitzeindex) describes the quantitative relationship between perceived temperature and humidity. According to wikipedia, it is applicable for temperatures above 26.7°C and 40 % relative humidity.

$$ \vartheta _{\mathrm {HI} }=c_{1}+c_{2}\vartheta +c_{3}\varphi +c_{4}\vartheta \varphi +c_{5}\vartheta ^{2}+c_{6}\varphi ^{2}+c_{7}\vartheta ^{2}\varphi +c_{8}\vartheta \varphi ^{2}+c_{9}\vartheta ^{2}\varphi ^{2}$$

|Parameter |	ϑ in °C |	ϑ in °F |
|----------|------------|-----------|
c1 | 	−8,784695 |	−42,379 |
c2 |	1,61139411 |	2,04901523 |
c3 |	2,338549 |	10,1433127 |
c4 |	−0,14611605 |	−0,22475541 |
c5 |	−1,2308094 · 10−2 |	−6,83783 · 10−3 |
c6 |	−1,6424828 · 10−2 |	−5,481717 · 10−2 |
c7 |	2,211732 · 10−3 |	1,22874 · 10−3 |
c8 |	7,2546 · 10−4 |	8,5282 · 10−4 |
c9 |	−3,582 · 10−6 |	−1,99 · 10−6 |

[Relative humidity](https://de.wikipedia.org/wiki/Luftfeuchtigkeit#Relative_Luftfeuchtigkeit) (German version of wikipedia is surprisingly better for this article) can be calculated based on the air pressure and the [saturation pressure of water in air](https://en.wikipedia.org/wiki/Vapour_pressure_of_water)

Another expression for perceived temperature is the [Humidex](https://en.wikipedia.org/wiki/Humidex) given by the formula:

$$
H = T_{air} + 0.5555 * (6.11^{5417.753 * (1/273.16 - 1/(273.15+T_{dew}))} - 10)
$$

$H$ ... Humidex \
$T_{air}$ ... Temperature of the air \
$T_{dew}$ ... Dew point of the air

The dew point can be calculated based on the Magnus Formula using the relative humidity of the air. For example formula 10 in the german wiki article about the [dew point](https://de.wikipedia.org/wiki/Taupunkt). Valid between -10 and 60°C.

$$
\tau(\varphi, \vartheta) = K_3 * \frac{\frac{K_2*\vartheta}{K_3 + \vartheta} + ln(\varphi)}{\frac{K_2*K_3}{K_3 + \vartheta}}
$$

https://labor.bht-berlin.de/fileadmin/labor/tvt/Umdrucke/2014_Trocknung-Schlempe.pdf

In [53]:
class constants:
    gas_constant_spec =  dict(water = 461.51,
                              air_dry = 287.058)  #  (J/(kg*K)
    gas_constant_general = 8314  # J/(kg*K)
    
    

def calc_dewpoint(humidity_rel: float, temp: float) -> float:
    """Calculates the dew point (taupunkt) of water in air based on the magnus formula

    Based on: https://de.wikipedia.org/wiki/Taupunkt ??

    Parameters
    ------
    humidity_rel: float
        relative humidity between 0 and 1
    temp: float
        temperature in °C
    
    Returns
    ------
    float
        dew point in °C
    """
    k3 = 243.12 # °C
    k2 = 17.62  # °C
    return k3 * (k2*temp/(k3+temp) + np.log(humidity_rel)) / (k2*k3 / (k3 + temp) - np.log(humidity_rel))

def calc_satpressure(temp: float) -> float:
    """Calcuates the saturation pressure of water in air based on the magnus formula
    
    https://www.eas.ualberta.ca/jdwilson/EAS372_13/Vomel_CIRES_satvpformulae.html

    Parameters:
    -----
    temp: float
        temperature in °C

    Returns:
    -----
        saturation pressure of water in Pa
    """
    k3 = 243.5 # °C
    k2 = 17.67  # °C
    return 610.78*np.exp(k2*temp / (k3 + temp))

def calc_sathumidity(temp: float, pressure_air: float) -> float:
    """Calculates the amount of water in saturated air as kg_water / kg_air_wet

    Parameters
    -----    
    temp: float
        temperature in °C
    
    pressure_air: float
        air pressure in Pa

    Returns
    ------
    float
        partial pressure of water in Pa
    """
    return 0.622 * calc_satpressure(temp) / (pressure_air - 0.378 * calc_satpressure(temp))

def calc_ppartial(humidity_rel:float, temp:float)->float:
    """Returns the partial pressure of water in Pa

    https://schweizer-fn.de/lueftung/feuchte/feuchte.php

    Parameters
    -----
    humidity_rel: float
        relative humidity between 0 and 1
    
    temp: float
        temperature in °C
    
    Returns
    ------
    float
        partial pressure of water in Pa
    """
    return humidity_rel * calc_satpressure(temp)


def calc_hum_spec(humidity_rel: float, temp: float, pressure_air: float) -> float:
    """Calculates speficic humidity between 0 and 1

    https://de.wikipedia.org/wiki/Luftfeuchtigkeit#Spezifische_Luftfeuchtigkeit
    https://schweizer-fn.de/lueftung/feuchte/feuchte.php#spezfeuchte

    Parameters
    ------
    temp: float 
        temperature in °C
    
    humidity_rel: float
        relative humidity between 0 and 1
    
    pressure_air: float
        air pressure in Pa

    Returns
    ------
    float
        specific humidity between 0 and 1
    """
    
    ppressure_water = calc_ppartial(humidity_rel, temp)
    rho_water = ppressure_water / (constants.gas_constant_spec["water"]*temp)
    rho_air_dry = (pressure_air - ppressure_water) / (constants.gas_constant_spec["air_dry"]*temp)
    return rho_water / (rho_air_dry + rho_water)

def calc_hum_abs(humidity_rel: float, temp: float, pressure_air: float) -> float:
    """Calculates humidity as kg_water / kg_air_dry
    https://de.wikipedia.org/wiki/Luftfeuchtigkeit#Spezifische_Luftfeuchtigkeit

    Parameters
    ------
    temp: float 
        temperature in °C
    
    humidity_rel: float
        relative humidity between 0 and 1
    
    pressure_air: float
        air pressure in Pa

    Returns
    -----
    float
        humidity as kg_water / kg_air_dry
    """
    ppartial_water = calc_ppartial(humidity_rel, temp)

    return 0.622 * ppartial_water / (pressure_air - ppartial_water)

def calc_humidex(temp: float, humidity_rel: float) -> float:
    """Calculates the humidex

    Source and possible test: https://en.wikipedia.org/wiki/Humidex
    Currently visually tested for dew point on x axis, not yet tested if dew point is calculated

    Parameters
    ----.
     temp: float 
        temperature in °C
    
    humidity_rel: float
        relative humidity between 0 and 1

    Returns
    ----
    float
        perceived temperature in °C
    """
    t_dew = calc_dewpoint(humidity_rel, temp)
    return temp + 0.5555 * (6.11*np.exp(5417.753 * (1/273.16 - 1/(273.15+t_dew))) - 10)

def calc_heat_evap(temp):
    """Calculates latent heat of evaporation for water in kg/kg
    https://de.wikipedia.org/wiki/Verdampfungsenthalpie

    Parameters
    ----.
     temp: float 
        temperature in °C

    Returns
    ----
    float
        latent heat of evaporation at given temperature in kj/kg
    """
    T = temp + 273.15
    return (50.09 - 0.9298 * T/1000 - 65.19 * (T/1000)**2) / 1800
    

    

Water mass can be calculated via

$$ ppartial_{water} = \varphi \, p_{sat} $$

$$ m_{water} = \frac{ppartial_{water}}{R_W \, T} $$

Partial pressure of water can be calculated accordingly as:

$$ ppartial_{water}  = \frac{m_{water}}{V_{total}} * R_W \, T $$

$$ V_{total} = V_{water} + V_{air} $$

$V_{water}$ is probably dependent on the temperature? For now I do ignore such possible dependencies.

In [31]:
calc_ppartial(0.5, 30) / 1e5 * 1000  # mbar

21.213289995023338

In [41]:
pressure = 1e5  # Pa
phi = 0.5
temp = 28
lenght = 4  # m
depth = 5  # m
height = 3  # m

volume_total = lenght * depth * height
ppartial_water = calc_ppartial(phi, temp)
rho_air = (pressure - ppartial_water) / (constants.gas_constant_spec["air_dry"]*(temp+273.15))
rho_water = ppartial_water / constants.gas_constant_spec["water"]
mu = calc_hum_abs(phi, temp, pressure)
m_air = volume_total  / (1/rho_air + mu/rho_water)  # kg
m_water = mu * m_air # kg

In [43]:
m_water

0.8128827355659307

In [44]:
m_water + m_air

68.68256197567216

In [46]:
calc_sathumidity(temp, pressure)

0.023842202976512867

In [49]:
# test if calculations are correct, should be equal to the assumed relative humidity (phi)
m_water / (calc_sathumidity(temp, pressure) * (m_water + m_air))

0.49640372550776485

In [50]:
calc_humidex(temp, phi)

33.01507578513029

In [64]:
# calculate new humidex after water was added
m_towel_dry = 1  # kg
factor_wet_dry = 3  # assumed factor describing m_towel_wet / m_towel_dry
m_water_evap = m_towel_dry * factor_wet_dry
m_water_new = m_water + m_water_evap


h_evap = calc_heat_evap(temp)  # kJ/kg

# TODO: cps are temperature dependent
cp_water = 4.2  # kJ/(kg*K)
cp_air_dry = 1  # kJ / (kg*K)

cp_air_wet = m_water / (m_water + m_air) * cp_water + m_air / (m_water + m_air) * cp_air_dry
# calculate new temperature

temp_new = temp - h_evap*m_water_evap / cp_air_wet
temp_new

27.929506761735695

In [78]:
# new humidex
hum_abs = m_water_new / (m_water_new + m_air)
phi_new =  hum_abs / calc_sathumidity(temp_new, pressure)
calc_humidex(temp_new, phi_new)

71.39569183593899

In [69]:
temp_new

27.929506761735695

In [70]:
phi_new

1.7019491400594409

In [71]:
calc_sathumidity(temp_new, pressure)

0.023742861487526818

In [76]:
hum_abs

0.05319121736831837

In [77]:
calc_sathumidity(temp_new, pressure)

0.023742861487526818

In [80]:
hum_abs

0.05319121736831837

In [81]:
calc_sathumidity(temp_new, pressure)

0.023742861487526818

In [82]:
m_water / (m_water + m_air)

0.011835358381853307